In all versions 5.x the narrative gets cleaned as before and products are cleaned three times. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time the same happens with the six most occuring ones. Finally, the components that almost never occur are removed. This run uses 10 different principle components. 

__Initialing the data__

In [2]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_both_scarce_elimination.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 5


__Initializing the bag of words__

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [4]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=10)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(203869, 10)


In [5]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.18481179 -3.602065    0.07901313 ... -0.55808707 -0.63825754
   0.78319113]
 [ 3.99935195 -1.16297815  0.68679863 ... -0.24819009 -0.10744953
  -0.14247453]
 [ 2.07097048  0.89951185  2.24984513 ...  0.014236    0.7256091
   0.63355153]
 ...
 [ 3.49460278 -3.52437615 -0.55894344 ... -0.50223818 -0.43054596
  -0.17712096]
 [ 2.46637635 -2.01364228 -1.34605033 ... -0.17156302 -0.87213809
  -0.22219866]
 [ 1.58335219 -1.22219907  0.0306219  ...  0.06473113 -0.21014603
  -0.04491385]]
                0          1         2         3          4         5  \
0        4.184812  -3.602065  0.079013  0.745045   0.011816  0.063925   
1        3.999352  -1.162978  0.686799 -0.619888   0.289567 -0.107508   
2        2.070970   0.899512  2.249845 -0.971209  -0.491737  0.208497   
3       24.403532 -15.836288  3.415882  8.222537 -10.634001  3.164170   
4        3.270027   0.836783  2.221457 -1.323723  -0.310826 -0.141760   
...           ...        ...       ...       ...        ...       ...  

__One hot encoding of extra columns__

In [6]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(203869, 10)
(203869, 149)
(203869, 159)


__Split into train and test__

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(152901, 159)
(50968, 159)
(152901,)
(50968,)


__Application of linear SVM__

In [8]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3632
                                                 Checking or savings account       0.99      1.00      1.00      4656
                                                               Consumer Loan       0.97      0.78      0.86      2958
                                                                 Credit card       1.00      1.00      1.00      4742
                                                 Credit card or prepaid card       0.96      0.96      0.96      4670
                                                            Credit reporting       1.00      1.00      1.00      4740
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.86      0.92      6189
                                                       